In [1]:
from langchain.schema import Document
import pdfplumber

def extract_text_as_documents(pdf_path):
    documents = []
    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            text = page.extract_text()
            if text:  # Avoid adding empty pages
                doc = Document(page_content=text, metadata={"page": i + 1, "source": pdf_path})
                documents.append(doc)
    return documents

In [2]:
docs = extract_text_as_documents("./docs/hon_nhan.pdf")
docs

[Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='16 CÔNG BÁO/Số 681 + 682/Ngày 16-7-2014\nQUỐC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\nLuật số: 52/2014/QH13\nLUẬT\nHÔN NHÂN VÀ GIA ĐÌNH\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;\nQuốc hội ban hành Luật hôn nhân và gia đình.\nChương I\nNHỮNG QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều chỉnh\nLuật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách\nứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước\nvà xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình.\nĐiều 2. Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình\n1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.\n2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người\ntheo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với người\nkhông có tín ngưỡng, giữa công dân Việt Nam với người nước ngoài được tô

## Recursive Splitter

In [3]:
separators = ["\n\n", "\n", " ", ""]
chunk_size = 300 
chunk_overlap = 20

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=separators,
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [5]:
chunks = splitter.split_documents(docs)
len(chunks), chunks

(347,
 [Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='16 CÔNG BÁO/Số 681 + 682/Ngày 16-7-2014\nQUỐC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\nLuật số: 52/2014/QH13\nLUẬT\nHÔN NHÂN VÀ GIA ĐÌNH\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;\nQuốc hội ban hành Luật hôn nhân và gia đình.\nChương I\nNHỮNG QUY ĐỊNH CHUNG'),
  Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='Điều 1. Phạm vi điều chỉnh\nLuật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách\nứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước\nvà xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình.'),
  Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='Điều 2. Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình\n1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.\n2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn 

In [6]:
print(chunks[0].page_content, len(chunks[0].page_content))
print("="*50)
print(chunks[1].page_content, len(chunks[1].page_content))
print("="*50)
print(chunks[2].page_content, len(chunks[2].page_content))

16 CÔNG BÁO/Số 681 + 682/Ngày 16-7-2014
QUỐC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM
Độc lập - Tự do - Hạnh phúc
Luật số: 52/2014/QH13
LUẬT
HÔN NHÂN VÀ GIA ĐÌNH
Căn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;
Quốc hội ban hành Luật hôn nhân và gia đình.
Chương I
NHỮNG QUY ĐỊNH CHUNG 292
Điều 1. Phạm vi điều chỉnh
Luật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách
ứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước
và xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình. 248
Điều 2. Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình
1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.
2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người
theo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với người 288


In [7]:
from langchain.embeddings.base import Embeddings
from langchain.vectorstores import FAISS, Chroma
from sentence_transformers import SentenceTransformer
from torch import embedding
from typing import Union, List


/Users/chessman/miniforge3/envs/PyJupyter/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
class SentenceTransformerEmbeddings(Embeddings):
    """Custom wrapper for SentenceTransformer to make it compatible with LangChain."""
    
    def __init__(self, model_name="hiieu/halong_embedding"):
        """Initialize the SentenceTransformer model."""
        self.model = SentenceTransformer(model_name)
        
    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        """Embed a list of documents using the SentenceTransformer model."""
        embeddings = self.model.encode(texts)
        return embeddings.tolist()
    
    def embed_query(self, text: str) -> List[float]:
        """Embed a query using the SentenceTransformer model."""
        embedding = self.model.encode([text])[0]
        return embedding.tolist()

In [9]:
embeddings = SentenceTransformerEmbeddings(model_name="hiieu/halong_embedding")

In [10]:
sentence1 = "luật hôn nhân"
sentence2 = "luật gia đình"
sentence3 = "thời tiết xấu"

In [11]:
import numpy as np
embedding1 = embeddings.embed_query(sentence1)
embedding2 = embeddings.embed_query(sentence2)
embedding3 = embeddings.embed_query(sentence3)


In [12]:
len(embedding1)

768

In [13]:
np.dot(embedding1, embedding2) # same 

0.7595786120760397

In [14]:
np.dot(embedding2, embedding3) # not same

0.1492165063993458

In [15]:
np.dot(embedding1, embedding3) # not same

0.216157058993724

In [16]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

In [17]:
vector_db._collection.count()

347

In [18]:
question = "hôn nhân là gì"

In [19]:
quest = vector_db.similarity_search(question,k=3)

In [20]:
quest

[Document(metadata={'page': 2, 'source': './docs/hon_nhan.pdf'}, page_content='lại trong một thời gian dài và được thừa nhận rộng rãi trong một vùng, miền hoặc\ncộng đồng.\n5. Kết hôn là việc nam và nữ xác lập quan hệ vợ chồng với nhau theo quy định\ncủa Luật này về điều kiện kết hôn và đăng ký kết hôn.\n6. Kết hôn trái pháp luật là việc nam, nữ đã đăng ký kết hôn tại cơ quan nhà'),
 Document(metadata={'page': 2, 'source': './docs/hon_nhan.pdf'}, page_content='12. Yêu sách của cải trong kết hôn là việc đòi hỏi về vật chất một cách quá\nđáng và coi đó là điều kiện để kết hôn nhằm cản trở việc kết hôn tự nguyện của\nnam, nữ.\n13. Thời kỳ hôn nhân là khoảng thời gian tồn tại quan hệ vợ chồng, được tính\ntừ ngày đăng ký kết hôn đến ngày chấm dứt hôn nhân.'),
 Document(metadata={'page': 2, 'source': './docs/hon_nhan.pdf'}, page_content='11. Kết hôn giả tạo là việc lợi dụng kết hôn để xuất cảnh, nhập cảnh, cư trú, nhập\nquốc tịch Việt Nam, quốc tịch nước ngoài; hưởng chế độ ưu đãi của Nhà nư

In [30]:
# Check Manual
matryoshka_dim = 300
model = SentenceTransformer(
    "hiieu/halong_embedding",
    truncate_dim=matryoshka_dim,
)

In [24]:
document_list = [doc.page_content for doc in chunks]

In [25]:
# Encode query and documents
query_embedding = model.encode([question])
doc_embeddings = model.encode(document_list)
similarities = model.similarity(query_embedding, doc_embeddings).flatten()

In [ ]:
len(similarities) # number of collections

In [ ]:
import torch

# Sort documents by cosine similarity
sorted_indices = torch.argsort(similarities, descending=True)
sorted_docs = [document_list[idx] for idx in sorted_indices[:3]]
sorted_scores = [similarities[idx].item() for idx in sorted_indices]

# Print sorted documents with their cosine scores
for doc, score in zip(sorted_docs, sorted_scores):
    print(f"Document: {doc}")
    print(f"Cosine Similarity: {score:.4f}")
    print("="*50)

## Semantic Splitter

In [21]:
from langchain_experimental.text_splitter import SemanticChunker

splitter = SemanticChunker(
    embeddings, breakpoint_threshold_type="percentile" # "standard_deviation", "interquartile"
)
chunks = splitter.split_documents(docs)
print(chunks)

[Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='16 CÔNG BÁO/Số 681 + 682/Ngày 16-7-2014\nQUỐC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\nLuật số: 52/2014/QH13\nLUẬT\nHÔN NHÂN VÀ GIA ĐÌNH\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;\nQuốc hội ban hành Luật hôn nhân và gia đình. Chương I\nNHỮNG QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều chỉnh\nLuật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách\nứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước\nvà xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình. Điều 2. Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình\n1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.'), Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người\ntheo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với n

In [22]:
len(chunks), chunks

(91,
 [Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='16 CÔNG BÁO/Số 681 + 682/Ngày 16-7-2014\nQUỐC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\nLuật số: 52/2014/QH13\nLUẬT\nHÔN NHÂN VÀ GIA ĐÌNH\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;\nQuốc hội ban hành Luật hôn nhân và gia đình. Chương I\nNHỮNG QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều chỉnh\nLuật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách\nứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước\nvà xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình. Điều 2. Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình\n1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.'),
  Document(metadata={'page': 1, 'source': './docs/hon_nhan.pdf'}, page_content='2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người\ntheo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡ

In [23]:
chunks[0].page_content

'16 CÔNG BÁO/Số 681 + 682/Ngày 16-7-2014\nQUỐC HỘI CỘNG HÒA XÃ HỘI CHỦ NGHĨA VIỆT NAM\nĐộc lập - Tự do - Hạnh phúc\nLuật số: 52/2014/QH13\nLUẬT\nHÔN NHÂN VÀ GIA ĐÌNH\nCăn cứ Hiến pháp nước Cộng hòa xã hội chủ nghĩa Việt Nam;\nQuốc hội ban hành Luật hôn nhân và gia đình. Chương I\nNHỮNG QUY ĐỊNH CHUNG\nĐiều 1. Phạm vi điều chỉnh\nLuật này quy định chế độ hôn nhân và gia đình; chuẩn mực pháp lý cho cách\nứng xử giữa các thành viên gia đình; trách nhiệm của cá nhân, tổ chức, Nhà nước\nvà xã hội trong việc xây dựng, củng cố chế độ hôn nhân và gia đình. Điều 2. Những nguyên tắc cơ bản của chế độ hôn nhân và gia đình\n1. Hôn nhân tự nguyện, tiến bộ, một vợ một chồng, vợ chồng bình đẳng.'

In [24]:
chunks[1].page_content

'2. Hôn nhân giữa công dân Việt Nam thuộc các dân tộc, tôn giáo, giữa người\ntheo tôn giáo với người không theo tôn giáo, giữa người có tín ngưỡng với người\nkhông có tín ngưỡng, giữa công dân Việt Nam với người nước ngoài được tôn\ntrọng và được pháp luật bảo vệ. 3. Xây dựng gia đình ấm no, tiến bộ, hạnh phúc; các thành viên gia đình có\nnghĩa vụ tôn trọng, quan tâm, chăm sóc, giúp đỡ nhau; không phân biệt đối xử\ngiữa các con. 4. Nhà nước, xã hội và gia đình có trách nhiệm bảo vệ, hỗ trợ trẻ em, người cao\ntuổi, người khuyết tật thực hiện các quyền về hôn nhân và gia đình; giúp đỡ các bà\nmẹ thực hiện tốt chức năng cao quý của người mẹ; thực hiện kế hoạch hóa gia đình. 5. Kế thừa, phát huy truyền thống văn hóa, đạo đức tốt đẹp của dân tộc Việt\nNam về hôn nhân và gia đình. Điều 3. Giải thích từ ngữ\nTrong Luật này, các từ ngữ dưới đây được hiểu như sau:\n1. Hôn nhân là quan hệ giữa vợ và chồng sau khi kết hôn. 2. Gia đình là tập hợp những người gắn bó với nhau do hôn nhân, quan hệ\nh

In [25]:
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings
)

In [26]:
vector_db._collection.count()

438

In [27]:
question = "hôn nhân là gì"

In [28]:
quest = vector_db.similarity_search(question,k=3)

In [29]:
quest

[Document(metadata={'page': 2, 'source': './docs/hon_nhan.pdf'}, page_content='lại trong một thời gian dài và được thừa nhận rộng rãi trong một vùng, miền hoặc\ncộng đồng.\n5. Kết hôn là việc nam và nữ xác lập quan hệ vợ chồng với nhau theo quy định\ncủa Luật này về điều kiện kết hôn và đăng ký kết hôn.\n6. Kết hôn trái pháp luật là việc nam, nữ đã đăng ký kết hôn tại cơ quan nhà'),
 Document(metadata={'page': 2, 'source': './docs/hon_nhan.pdf'}, page_content='Thời kỳ hôn nhân là khoảng thời gian tồn tại quan hệ vợ chồng, được tính\ntừ ngày đăng ký kết hôn đến ngày chấm dứt hôn nhân. 14. Ly hôn là việc chấm dứt quan hệ vợ chồng theo bản án, quyết định có hiệu\nlực pháp luật của Tòa án. 15. Ly hôn giả tạo là việc lợi dụng ly hôn để trốn tránh nghĩa vụ tài sản, vi\nphạm chính sách, pháp luật về dân số hoặc để đạt được mục đích khác mà không\nnhằm mục đích chấm dứt hôn nhân.'),
 Document(metadata={'page': 2, 'source': './docs/hon_nhan.pdf'}, page_content='12. Yêu sách của cải trong kết hô